In [ ]:
using Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")
Pkg.add("Plots")
Pkg.add("Combinatorics")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed StructTypes ──────── v1.11.0
   Installed CodecBzip2 ───────── v0.8.5
   Installed JSON3 ────────────── v1.14.3
   Installed BenchmarkTools ───── v1.6.3
   Installed MutableArithmetics ─ v1.6.7
   Installed MathOptInterface ─── v1.46.0
   Installed JuMP ─────────────── v1.29.2
    Updating `~/.julia/environments/v1.11/Project.toml`
  [4076af6c] + JuMP v1.29.2
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
  [523fee87] + CodecBzip2 v0.8.5
  [0f8b85d8] + JSON3 v1.14.3
  [4076af6c] + JuMP v1.29.2
  [b8f27783] + MathOptInterface v1.46.0
  [d8a4904e] + MutableArithmetics v1.6.7
  [856f2bd8] + StructTypes v1.11.0
  [9abbd945] + Profile v1.11.0
Precompiling project...
   6451.7 ms  ✓ CodecBzip2
   7964.2 ms  ✓ StructTypes
   4058.3 ms  ✓ BenchmarkTools
  18444.6 ms  ✓ MutableArithmetics
  20762.7 ms  ✓ JSON3
  82784.4 ms  ✓ MathOptInterface
 

In [ ]:
using JuMP, HiGHS
using Plots
using LinearAlgebra
using Random
using Combinatorics

#MODELO JUGUETE

Solucion ={capacidad del vehiculo, ruta}

{50,{1,3,4,7}}

{18,{2}}

{9,{5}}

{5,{6}}

In [ ]:
#TOY MODEL
n_camiones = 4
n_clientes = 7
capacidades = [50,18,9,5] #solo 4 camiones
demandas_clientes=[10,15,10,10,7,4,20] #solo 7 clientes
c_clientes = collect(1:n_clientes) #Conjunto de clientes
PJ = [collect(c) for k in 1:length(c_clientes) for c in combinations(c_clientes, k)] #Conjunto potencia de clientes
card_PJ = length(PJ) #Número de rutas segun ruta de clientes
dj = zeros(card_PJ) #valores dj del problema
for i in 1:card_PJ
  sum=0
  for j in PJ[i]
    sum+=demandas_clientes[j]
  end
  dj[i] = sum
end
indice_ruta1 = findfirst(x -> x == [1,3,4,7], PJ)
indice_ruta2 = findfirst(x -> x == [1,3,4,7], PJ)
indice_ruta3 = findfirst(x -> x == [1,3,4,7], PJ)
indice_ruta4 = findfirst(x -> x == [1,3,4,7], PJ)

rj = ones(card_PJ)*10000
rj[indice_ruta1]=30
rj[indice_ruta1]=20
rj[indice_ruta1]=15
rj[indice_ruta1]=40

#Simulamos la funcion indicadora
S_R = zeros(n_camiones,card_PJ)
for i in 1:n_camiones
  for j in 1:card_PJ
    S_R[i,j] = capacidades[i] + rj[j]
  end
end

Ik = zeros(n_clientes,card_PJ)
for i in 1:n_clientes
  for j in 1:card_PJ
    Ik[i,j] = Int(i in PJ[j])
  end
end

M = Model(HiGHS.Optimizer)
@variable(M, x[1:n_camiones, 1:card_PJ] >= 0, Bin)
@objective(M, Min, sum(x[i,j] * S_R[i,j] for i in 1:n_camiones, j in 1:card_PJ))
@constraint(M, [i in 1:n_camiones], sum(x[i,j] for j in 1:card_PJ) <= 1)
@constraint(M, [i in 1:n_camiones], sum(x[i,j]*dj[j] for j in 1:card_PJ) <= capacidades[i])
@constraint(M, [k in 1:n_clientes], sum(x[i,j]*Ik[k,j] for i in 1:n_camiones,j in 1:card_PJ) == 1)
optimize!(M)

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 15 rows; 508 cols; 2808 nonzeros; 508 integer variables (508 binary)
Coefficient ranges:
  Matrix  [1e+00, 8e+01]
  Cost    [4e+01, 1e+04]
  Bound   [1e+00, 1e+00]
  RHS     [1e+00, 5e+01]
Presolving model
14 rows, 113 cols, 543 nonzeros  0s
11 rows, 52 cols, 196 nonzeros  0s
10 rows, 19 cols, 69 nonzeros  0s
9 rows, 17 cols, 53 nonzeros  0s
Presolve reductions: rows 9(-6); columns 17(-491); nonzeros 53(-2755) 
Objective function is integral with scale 1

Solving MIP model with:
   9 rows
   17 cols (17 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   53 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Triv

Presentando la solucion:

In [ ]:
z_opt = objective_value(M)
X_opt = value.(x)

rutas_posiciones = sum(X_opt, dims=1) #Suma de columnas
vehiculos_empleados = round(Int,sum(sum(X_opt, dims =2)))

rutas_solucion = []

for i in 1:card_PJ
  if rutas_posiciones[i]>0.001
    for j in 1:n_camiones
      if X_opt[j,i]>0.001
        push!(rutas_solucion,[j,PJ[i]])
      end
    end
  end
end

println("Se emplearon ",vehiculos_empleados," vehiculos")
for i in rutas_solucion
  println("El vehiculo ",i[1]," para por los clientes: ",i[2])
end

Se emplearon 4 vehiculos
El vehiculo 2 para por los clientes: [2]
El vehiculo 3 para por los clientes: [5]
El vehiculo 4 para por los clientes: [6]
El vehiculo 1 para por los clientes: [1, 3, 4, 7]


#PROBLEMA CON DATOS GENERADOS ALEATORIAMENTE

In [ ]:
##########################################
#Capacidades de los camiones
n_camiones =30 #Numero de camiones
tipos_camiones=[0.65,0.15,0.05,0.15] #Proporciones de camiones
capacidades_camiones = [450,800,1400,2000] #Capacidades por tipo de camiones
##########################################

n_tipo_camion=zeros(Int,length(tipos_camiones))
for i in 1:(length(tipos_camiones)-1)
  n_tipo_camion[i]=round(Int,n_camiones*tipos_camiones[i])
end
n_tipo_camion[length(tipos_camiones)] = n_camiones-sum(n_tipo_camion) #Numero de camiones por tipo

capacidades = reduce(vcat, [fill(capacidades_camiones[i], n_tipo_camion[i]) for i in eachindex(n_tipo_camion)]); #Capacidades de cada camion

##########################################
#Demandas de clientes
Random.seed!(123) #fijar la semilla del generador de números aleatorios
n_clientes = 100 #Numero de clientes
c_clientes = collect(1:n_clientes) #Conjunto de clientes
demandas_clientes = rand(200:600, n_clientes) #Generando demandas de clientes

100-element Vector{Int64}:
 409
 435
 557
 276
 410
 356
 217
 574
 432
 331
 411
 535
 216
   ⋮
 449
 333
 482
 556
 460
 234
 260
 272
 488
 548
 242
 520

cuello de botella:

In [ ]:
demandas_sort =  sort(demandas_clientes)
println(demandas_sort)
demandas_acum = cumsum(demandas_sort)
println(demandas_acum)
max_capacity = maximum(capacidades)
n_max_comb = 0
for i in 1:length(demandas_acum)
  if demandas_acum[i]>max_capacity
      n_max_comb = i-1
    break
  end
end
println("Maxima capacidad: ",max_capacity)
println("Numero maximo de combinaciones: ",n_max_comb)
println("Numero de combinacion: ",binomial(n_clientes,n_max_comb))

[207, 216, 217, 218, 223, 228, 228, 234, 235, 236, 238, 240, 241, 242, 246, 248, 252, 260, 262, 272, 276, 278, 282, 282, 289, 293, 306, 310, 318, 318, 324, 325, 331, 333, 345, 350, 351, 352, 355, 355, 356, 367, 375, 378, 385, 386, 386, 409, 410, 411, 417, 425, 430, 432, 432, 433, 434, 435, 435, 449, 449, 456, 456, 460, 466, 469, 471, 479, 482, 483, 483, 486, 488, 488, 493, 509, 511, 518, 518, 520, 522, 524, 530, 535, 537, 544, 546, 548, 551, 552, 553, 556, 557, 561, 566, 569, 574, 574, 594, 598]
[207, 423, 640, 858, 1081, 1309, 1537, 1771, 2006, 2242, 2480, 2720, 2961, 3203, 3449, 3697, 3949, 4209, 4471, 4743, 5019, 5297, 5579, 5861, 6150, 6443, 6749, 7059, 7377, 7695, 8019, 8344, 8675, 9008, 9353, 9703, 10054, 10406, 10761, 11116, 11472, 11839, 12214, 12592, 12977, 13363, 13749, 14158, 14568, 14979, 15396, 15821, 16251, 16683, 17115, 17548, 17982, 18417, 18852, 19301, 19750, 20206, 20662, 21122, 21588, 22057, 22528, 23007, 23489, 23972, 24455, 24941, 25429, 25917, 26410, 26919, 27430,

In [ ]:
Random.seed!(123) #fijar la semilla del generador de números aleatorios
n_clientes = 15 #Numero de clientes
c_clientes = collect(1:n_clientes) #Conjunto de clientes
demandas_clientes = rand(200:600, n_clientes) #Generando demandas de clientes

PJ = [collect(c) for k in 1:length(c_clientes) for c in combinations(c_clientes, k)] #Conjunto potencia de clientes
card_PJ = length(PJ) #Número de rutas segun ruta de clientes
dj = zeros(card_PJ) #valores dj del problema
for i in 1:card_PJ
  sum=0
  for j in PJ[i]
    sum+=demandas_clientes[j]
  end
  dj[i] = sum
end
rj = rand(500:800, card_PJ); #Vector del costo de cada ruta de clientes


La funcion objetivo $\sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{P}(\mathcal{J})} x_{i,j}(s_{i}+r_{j})$ la podemos expresar como la suma de los elementos del producto elemento a elemento de las matrices: $[x_{i,j}]_{n(\mathcal{I}) \times n(P(\mathcal{J}))}$ y $[s_{i}+r_{j}]_{n(\mathcal{I}) \times n(P(\mathcal{J}))}$, y lo denotamos como:

$$ sum([x_{i,j}] \circ [s_{i}+r_{j}])=tr([x_{i,j}]^{T} [s_{i}+r_{j}])$$

A la matriz $[s_{i}+r_{j}]_{n(\mathcal{I}) \times n(P(\mathcal{J}))}$ la denotamos como S_R

In [ ]:
S_R = zeros(n_camiones,card_PJ)
for i in 1:n_camiones
  for j in 1:card_PJ
    S_R[i,j] = capacidades[i] + rj[j]
  end
end

Ik = zeros(n_clientes,card_PJ)
for i in 1:n_clientes
  for j in 1:card_PJ
    Ik[i,j] = Int(i in PJ[j])
  end
end

println(size(Ik[2,:]))

Resolviendo el problema

In [ ]:
M = Model(HiGHS.Optimizer)
@variable(M, x[1:n_camiones, 1:card_PJ] >= 0, Bin)
@objective(M, Min, sum(x[i,j] * S_R[i,j] for i in 1:n_camiones, j in 1:card_PJ))
@constraint(M, [i in 1:n_camiones], sum(x[i,j] for j in 1:card_PJ) <= 1)
@constraint(M, [i in 1:n_camiones], sum(x[i,j]*dj[j] for j in 1:card_PJ) <= capacidades[i])
@constraint(M, [k in 1:n_clientes], sum(x[i,j]*Ik[k,j] for i in 1:n_camiones,j in 1:card_PJ) == 1)
optimize!(M)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 55 rows; 655340 cols; 6225880 nonzeros; 655340 integer variables (655340 binary)
Coefficient ranges:
  Matrix [1e+00, 6e+03]
  Cost   [1e+03, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 8e+02]
Presolving model
55 rows, 505 cols, 1777 nonzeros  12s
55 rows, 505 cols, 1777 nonzeros  12s
Objective function is integral with scale 1

Solving MIP model with:
   55 rows
   505 cols (505 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   1777 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds    

In [ ]:
z_opt = objective_value(M)
X_opt = value.(x)
println("Valor óptimo de la función objetivo = ", z_opt)
println("Solución óptima de X =")
display(X_opt)

Valor óptimo de la función objetivo = 12721.000000000004
Solución óptima de X =


20×32767 Matrix{Float64}:
  0.0  0.0           0.0  0.0          …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  1.0           0.0  0.0             0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0           0.0  6.04492e-15     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0           0.0  0.0             0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0           0.0  0.0             0.0  0.0  0.0  0.0  0.0  0.0  0.0
  1.0  0.0           0.0  0.0          …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0           0.0  0.0             0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0           0.0  0.0             0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0           0.0  0.0             0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  1.42109e-14   0.0  0.0             0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0           0.0  0.0          …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0  0.0           0.0  0.0             0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0           0.0  0.0             0.0  0.0  0.0  0.0  

In [ ]:
rutas_posiciones = sum(X_opt, dims=1) #Suma de columnas
vehiculos_empleados = round(Int,sum(sum(X_opt, dims =2)))

rutas_solucion = []

for i in 1:card_PJ
  if rutas_posiciones[i]>0.001
    for j in 1:n_camiones
      if X_opt[j,i]>0.001
        push!(rutas_solucion,[j,PJ[i]])
      end
    end
  end
end

println("Se emplearon ",vehiculos_empleados," vehiculos")
for i in rutas_solucion
  println("El vehiculo ",i[1]," para por los clientes: ",i[2])
end

Se emplearon 11 vehiculos
El vehiculo 6 para por los clientes: [1]
El vehiculo 2 para por los clientes: [2]
El vehiculo 13 para por los clientes: [5]
El vehiculo 5 para por los clientes: [9]
El vehiculo 8 para por los clientes: [10]
El vehiculo 11 para por los clientes: [11]
El vehiculo 15 para por los clientes: [12]
El vehiculo 20 para por los clientes: [3, 7]
El vehiculo 17 para por los clientes: [4, 6]
El vehiculo 18 para por los clientes: [8, 13]
El vehiculo 19 para por los clientes: [14, 15]


#Otra posible solucion (falta completar)

Reformularemos el problema:

$$\sum_{i \in \mathcal{I}} \sum_{i \in \mathcal{R}_{i}} x_{i,j}(s_i+r_j): \sum_{j \in \mathcal{R}_{i}} x_{i,j} \leq 1;i\in \mathcal{I}; \sum_{j \in \mathcal{R}_{i}} x_{i,j}d_{j} \leq s_i;i\in \mathcal{I}; \sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{R}_{i}} x_{i,j}I_{k \in j} =  1, k\in \mathcal{J}   $$

Donde:
$R_i \in \mathcal{P} (\mathcal{J})$, es el conjunto de rutas factibles del camion $i$, así retiramos la segunda restriccion del problema ya que siempre se cumplira.

Debemos generar un conjunto con las posibles rutas para cada camion, el cual dependerá de las capacidades de cada camion y que las rutas no superen la capacidad de cada camion.

In [ ]:
idx = sortperm(dj) #indices ordenados de las demandas de cada ruta
dj_sort = dj[idx]
#println(dj_sort)
#println(PJ[idx])

1048575-element Vector{Float64}:
  216.0
  217.0
  240.0
  276.0
  318.0
  331.0
  345.0
  356.0
  386.0
  409.0
  410.0
  411.0
  430.0
    ⋮
 7422.0
 7423.0
 7446.0
 7476.0
 7487.0
 7501.0
 7514.0
 7556.0
 7592.0
 7615.0
 7616.0
 7832.0

In [ ]:
R_i = []
for i in 1:n_camiones
rutas =[]
  for j in 1:card_PJ
    if dj_sort[j]>capacidades[i]
      break
    end
    push!(rutas,idx[j])
  end
  push!(R_i, rutas)
end

In [ ]:
M = Model(HiGHS.Optimizer)

x = Dict{Tuple{Int, Int}, VariableRef}()

for i in 1:n_camiones
  for j in 1:(length(R_i[i]))
  #@variable(M, x[1:n_camiones, 1:(length(R_i[i]))] >= 0, Bin)
    x[(i,j)] = @variable(M, binary = true, base_name = "x_$(i)_$(j)")
  end
end

@objective(M, Min, sum(x[(i, j)]*S_R[i,R_i[i][j]] for (i,j) in keys(x)))

for i in 1:n_camiones
  @constraint(M, sum(x[(i, j)] for j in 1:(length(R_i[i]))) <= 1)
end

for k in 1:n_clientes
  @constraint(M, sum(sum(x[(i, j)]*Ik[k,R_i[i][j]]  for j in 1:(length(R_i[i]))) for i in 1:n_camiones) == 1)
end


#@constraint(M, [k in 1:n_clientes], sum(x[i,j]*Ik[k,j] for i in 1:n_camiones,j in 1:card_PJ) == 1)
print(M)
optimize!(M)


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 40 rows; 778 cols; 2042 nonzeros; 778 integer variables (778 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+03, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
40 rows, 778 cols, 2042 nonzeros  0s
40 rows, 778 cols, 2042 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   40 rows
   778 cols (778 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   2042 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  D

In [ ]:
for (k, v) in x
  if value(v)>0.001
    println("vehiculo: ",k[1],",ruta: ",PJ[R_i[k[1]][k[2]]])
  end
end

vehiculo: 18,ruta: [4, 17]
vehiculo: 1,ruta: [14]
vehiculo: 6,ruta: [2]
vehiculo: 8,ruta: [9]
vehiculo: 14,ruta: [12]
vehiculo: 13,ruta: [1]
vehiculo: 20,ruta: [3, 16]
vehiculo: 17,ruta: [6, 10]
vehiculo: 5,ruta: [5]
vehiculo: 19,ruta: [7, 8]
vehiculo: 15,ruta: [13, 20]
vehiculo: 10,ruta: [11]
vehiculo: 3,ruta: [15]
vehiculo: 11,ruta: [18]
vehiculo: 16,ruta: [19]


#Knapsack Problem

In [ ]:
#Datos del problema
valores = [60,100,120] #valores decada objeto
pesos = [10,20,30] #pesos de cada objeto
W = 50 #Capacidad de la mochila
n = length(valores) #Cantidad de obtjetos

#Creamos una matriz dp de dimensiones (n+1)*(W+1)
# cada valor dp[i+1,w+1] es el maximo valor usando los primeros objetos con capacidad w
dp = zeros(Int, n+1,W+1)

4×51 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [ ]:
#Llenado de la tabla
for i in 1:n
  for w in 0:W
    if pesos[i] <= w
      #Se elige el maximo entre:
      # - no tomar el objeto i
      # - tomarlo (valor + valor optimo con capacidad restante)
      dp[i+1,w+1] = max(dp[i,w+1], valores[i] + dp[i,w - pesos[i]+1])
    else
      #No se puede tomar el objeto i
      dp[i+1,w+1] = dp[i, w+1]
    end
  end
end

println("Valor maximo alcanzable: ", dp[n+1,W+1])
